# Redukowanie macierzy
## Konrad Linkowski

### Klasa Number

In [1]:
class Number:
    numerator = 0
    denominator = 1
    
    def __init__(self, num, denominator = 1):
        if isinstance(num, Number):
            self.numerator = num.numerator
            self.denominator = num.denominator
        else:
            self.numerator = num
            self.denominator = denominator
    
    def __add__(self, num):
        lcm = Number.lcm(self.denominator, num.denominator)
        return Number(self.numerator * lcm / self.denominator + num.numerator * lcm / num.denominator,
            Number.lcm(self.denominator, num.denominator)).normalize()
    
    def __mul__(self, num):
        return Number(self.numerator * num.numerator, self.denominator * num.denominator).normalize()
    
    def __eq__(self, num):
        a = self.normalize()
        b = num.normalize()
        return a.numerator == b.numerator and a.denominator == b.denominator
    
    def __str__(self):
        if self.denominator == 1: return str(int(self.numerator))
        return "{} / {}".format(int(self.numerator), int(self.denominator))
    
    def normalize(self):
        gcd = Number.gcd(self.numerator, self.denominator)
        num = self.numerator / gcd
        den = self.denominator / gcd
        if den < 0:
            num *= -1
            den *= -1
        return Number(num, den)
    
    def inverse(self):
        return Number(self.denominator, self.numerator)
    
    def opposite(self):
        return Number(-self.numerator, self.denominator)
    
    def clone(self):
        return Number(self.numerator, self.denominator)
    
    @staticmethod
    def gcd(a, b):
        aa = a
        bb = b
        while bb != 0:
            c = aa % bb
            aa = bb
            bb = c
        return aa
    
    @staticmethod
    def lcm(a, b):
        return a / Number.gcd(a, b) * b

### Klasa Matrix

In [2]:
class Matrix:
    rows = 0
    columns = 0
    values = []
    
    def __init__(self, rows = 0, columns = 0, *values):
        self.rows = rows
        self.columns = columns
        if len(values) != 0:
            self.values = [[Number(values[y * columns + x]) for x in range(columns)] for y in range(rows)] 
        else:
            self.values = [[Number(0) for x in range(columns)] for y in range(rows)] 
    
    def reduce(self):
        mat = self.clone()
        for i in range(mat.columns):
            for j in range(i, mat.rows):
                if not(mat.get(j, i) == Number(0)):
                    mat = mat.swapRows(j, i).multiplyRow(i, mat.get(j, i).inverse())
                    break
            for k in range(i + 1, mat.rows):
                if not(mat.get(k, i) == Number(0)):
                    mat = mat.multiplyRow(k, mat.get(k, i).inverse().opposite()).addRows(k, i)
        return mat
    
    def multiplyRow(self, row, factor):
        mat = self.clone()
        for i in range(mat.columns):
            mat.set(row, i, mat.get(row, i) * factor)
        return mat
    
    def addRows(self, a, b):
        mat = self.clone()
        for i in range(mat.columns):
            mat.set(a, i, mat.get(a, i) + mat.get(b, i))
        return mat
    
    def swapRows(self, a, b):
        mat = self.clone()
        if a == b:
            return mat
        for i in range(mat.columns):
            temp = mat.get(a, i)
            mat.set(a, i, mat.get(b, i))
            mat.set(b, i, temp)
        return mat
    
    def get(self, row, column):
        return self.values[row][column]
    
    def set(self, row, column, value):
        self.values[row][column] = value
        return self
    
    def clone(self):
        mat = Matrix(self.rows, self.columns)
        mat.values = [[self.get(y, x).clone() for x in range(mat.columns)] for y in range(mat.rows)]
        return mat
    
    def __str__(self):
        result = ""
        for i in range(self.rows):
            for j in range(self.columns):
                result +=str(self.values[i][j]) + "\t"
            result += "\n"
        return result

### Zastosowanie

In [3]:
mat = Matrix(4, 4,
             1, 3, 0, -1,
             0, 2, 1, 3,
             3, 1, 2, 1,
             -1, 2, 0, 3
            )
print(mat)
print(mat.reduce())

1	3	0	-1	
0	2	1	3	
3	1	2	1	
-1	2	0	3	

1	3	0	-1	
0	1	1 / 2	3 / 2	
0	0	1	8 / 3	
0	0	0	1	

